# Community Question Answering (Subtask A)

In [0]:
from __future__ import print_function

from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.utils import *
from keras import backend as K
from keras.engine.topology import Layer
from keras.callbacks import ModelCheckpoint
import time
import numpy as np
import json
import h5py
from google.colab import files

In [0]:
import tensorflow, keras
print("Tensorflow version ", tensorflow.__version__)
print("Keras version ", keras.__version__)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Tensorflow version  1.8.0
Keras version  2.1.6


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12573320392879952458, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11287966516
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18361824039973930569
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

## Acquire Dataset

In [0]:
# download and unzip embeddings and dataset
!wget -O glove.zip http://nlp.stanford.edu/data/glove.6B.zip
!unzip -o glove.zip -d glove
!wget -O task3-cqa.zip http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-cqa-ql-traindev-v3.2.zip
!unzip -o task3-cqa.zip -d data
!wget -O test.zip http://alt.qcri.org/semeval2017/task3/data/uploads/semeval2017_task3_test.zip
#!wget -O test.zip http://alt.qcri.org/semeval2017/task3/data/uploads/semeval2017_task3_test_input_abcd.zip
!unzip -o test.zip -d test_data

--2018-06-10 16:10:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-06-10 16:10:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.zip’

glove.zip           100%[===================>] 822.24M  31.9MB/s    in 23s     

2018-06-10 16:11:02 (35.5 MB/s) - ‘glove.zip’ saved [862182613/862182613]

Archive:  glove.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  
--2018-06-10 16:11:28--  http://alt.qcri.org/semeval2016/task3/data/uploads/s


  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part1-subtaskA-with-multiline.xml  
  inflating: data/v3.2/train/SemEval2016-Task3-CQA-QL-train-part1-subtaskA.xml  
  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part1-subtaskA.xml  
  inflating: data/v3.2/train/SemEval2016-Task3-CQA-QL-train-part1-with-multiline.xml  
  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part1-with-multiline.xml  
  inflating: data/v3.2/train/SemEval2016-Task3-CQA-QL-train-part1.xml  
  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part1.xml  
  inflating: data/v3.2/train/SemEval2016-Task3-CQA-QL-train-part2-subtaskA-with-multiline.xml  
  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part2-subtaskA-with-multiline.xml  
  inflating: data/v3.2/train/SemEval2016-Task3-CQA-QL-train-part2-subtaskA.xml  
  inflating: data/__MACOSX/v3.2/train/._SemEval2016-Task3-CQA-QL-train-part2-subtaskA.xml  
  i

In [0]:
ls *

glove.zip  task3-cqa.zip  test.zip

data:
__MACOSX/  v3.2/

datalab:

glove:
glove.6B.100d.txt  glove.6B.200d.txt  glove.6B.300d.txt  glove.6B.50d.txt

test_data:
__MACOSX/  SemEval2017_task3_test/


In [0]:
train_dir = 'data/v3.2/train/'
dev_dir = 'data/v3.2/dev/'
test_dir = 'test_data/SemEval2017_task3_test/English/'

## Preprocessing

In [0]:
class MapHelper:
    
    def append_to_list(map_with_list_as_value, map_key, new_list_element):
        existing_list = map_with_list_as_value.get(map_key, [])
        new_list = existing_list + [new_list_element]
        map_with_list_as_value[map_key] = new_list
    
    def append_to_set(map_with_set_as_value, map_key, new_set_element):
        existing_set = map_with_set_as_value.get(map_key, set())
        existing_set.add(new_set_element)
        map_with_set_as_value[map_key] = existing_set

In [0]:
class AnswerRelevance:
    """
        Makes no distinction between 
        PotentiallyUseful and Bad comments
    """
    
    INT_GOOD = 1
    GOOD = "Good"
    
    POTENTIALLY_USEFUL = "PotentiallyUseful"
    
    INT_BAD = 0
    BAD = "Bad"
    
    
    def to_number(string_relevance):
        return AnswerRelevance.INT_GOOD \
            if string_relevance == AnswerRelevance.GOOD \
            else AnswerRelevance.INT_BAD
    
    
    def from_number(int_relevance):
        return AnswerRelevance.GOOD \
            if string_relevance == AnswerRelevance.INT_GOOD \
            else AnswerRelevance.BAD

In [0]:
class Thread:
    
    def __init__(self, question, answers):
        self.question = question
        self.answers = answers

    def __str__(self):
        return self.question.__str__()
    
    def __repr__(self):
        return self.question.__repr__()

In [0]:
class ThreadCollection:
    
    def __init__(self):
        self.by_topic = {}
    
    def add(self, string_topic, thread):
        MapHelper.append_to_list(self.by_topic, string_topic, thread)
    
    def get(self, string_topic):
        return self.by_topic.get(string_topic, [])
    
    def topics(self):
        return list(self.by_topic.keys())

    def topic_thread_pairs(self):
        return self.by_topic.items()

In [0]:
from collections import namedtuple


class QuestionData:
    
    STRING_CATEGORY_KEY = "RELQ_CATEGORY"
    STRING_ID_KEY = "RELQ_ID"
    
    def __init__(self, id, subject, body):
        self.id = id
        self.subject = subject
        self.body = body
    
    def __str__(self):
        return str(self.body)
    
    def __repr__(self):
        return self.__str__()
        
        
class AnswerData:
    
    STRING_RELEVANCE_KEY = "RELC_RELEVANCE2RELQ"
    STRING_USERNAME_KEY = "RELC_USERNAME"
    STRING_DATE_KEY = "RELC_DATE"
    STRING_ID_KEY = "RELC_ID"
    
    def __init__(self, metadata, body):
        self.metadata = metadata
        self.body = body
        
    def __str__(self):
        return str(self.body)
    
    def __repr__(self):
        return self.__str__()

    
class Metadata:
    
    def __init__(self, id, user, date, relevance):
        self.id = id
        self.user = user
        self.date = date
        self.relevance = relevance
    
    
QuestionAnswerPair = namedtuple('QuestionAnswerPair', ['question', 'answer', 'data'])

# set data to None by default
QuestionAnswerPair.__new__.__defaults__ = (None,)

In [0]:
def to_question_data(node_question):
    subject = node_question[0]
    body = node_question[1]
    
    return QuestionData(node_question.attrib[QuestionData.STRING_ID_KEY], subject.text, body.text)


def to_answer_data(node_thread_answers):
    f = lambda node: AnswerData(Metadata(node.attrib[AnswerData.STRING_ID_KEY],
                                         node.attrib[AnswerData.STRING_USERNAME_KEY],
                                         node.attrib[AnswerData.STRING_DATE_KEY],
                                         AnswerRelevance.to_number(node.attrib[AnswerData.STRING_RELEVANCE_KEY])), 
                                node[0].text)
    
    return [f(node) for node in node_thread_answers]

In [0]:
from nltk.corpus.reader import XMLCorpusReader, XMLCorpusView

def load_data(string_basedir, string_filenames_in_basedir, subtask_A=False):
    for string_filename in string_filenames_in_basedir:
        corpus_reader = XMLCorpusReader(string_basedir, string_filename)
        
        for root_node in corpus_reader.xml():
            if not subtask_A:
                node_thread = root_node[2]
            else:
                node_thread = root_node
            node_thread_question = node_thread[0]
            node_thread_answers = node_thread[1:]

            thread_question = to_question_data(node_thread_question)
            thread_answers = to_answer_data(node_thread_answers)

            thread = Thread(thread_question, thread_answers)        
            string_topic = node_thread_question.attrib[QuestionData.STRING_CATEGORY_KEY]
            
            yield string_topic, thread

In [0]:
from itertools import *


def get_limited_generator(string_basedir,
                          list_string_filenames_in_basedir,
                          start_index_inclusive, 
                          end_index_exclusive = None, 
                          step = None,
                          subtask_A = False):
    data_generator = load_data(string_basedir, list_string_filenames_in_basedir, subtask_A)
    return islice(data_generator, start_index_inclusive, end_index_exclusive, step)

In [0]:
def repeat_relevant(question, list_relevant, int_how_many):
    for i in range(int_how_many):
        question = QuestionData(str(question.id), str(question.subject), str(question.body))
        yield QuestionAnswerPair(question, list_relevant[i % len(list_relevant)] )
    

def get_data(data_generator, do_repeat_relevant = True):
    X = []
    y = []
    
    for topic, thread in data_generator:
        question = thread.question
        list_relevant = []
        
        for answer in thread.answers:
            question = QuestionData(str(question.id), str(question.subject), str(question.body))
            question_answer_pair = QuestionAnswerPair(question, answer)
            
            X += [ question_answer_pair ]
            y += [ answer.metadata.relevance ]
            
            if answer.metadata.relevance == AnswerRelevance.INT_GOOD:
                list_relevant += [answer]
        
        num_relevant = len(list_relevant)
        num_irrelevant = len(thread.answers) - num_relevant
        
        if num_relevant < num_irrelevant \
            and do_repeat_relevant \
            and num_relevant != 0:
                
            delta = num_irrelevant - num_relevant
            
            X += repeat_relevant(question, list_relevant, delta)
            y += [ AnswerRelevance.INT_GOOD for i in range(delta) ]
        
    return X, y

## Dataset Loading

In [0]:
train_generator = get_limited_generator(train_dir, ['SemEval2016-Task3-CQA-QL-train-part1.xml', 'SemEval2016-Task3-CQA-QL-train-part2.xml'], 
                                       0, None, None)

dev_generator = get_limited_generator(dev_dir, ['SemEval2016-Task3-CQA-QL-dev.xml'], 0, None, None)
test_generator = get_limited_generator(test_dir, ['SemEval2017-task3-English-test.xml'], 0, None, None)
test_generator_A = get_limited_generator(test_dir, ['SemEval2017-task3-English-test-subtaskA.xml'], 0, None, None, subtask_A=True)

X = {"train": [], "dev":[], "test":[], "test_A":[]}
y = {"train": [], "dev":[], "test":[], "test_A":[]}

X["train"], y["train"] = get_data(train_generator, do_repeat_relevant=False)
X["dev"], y["dev"] = get_data(dev_generator, do_repeat_relevant=False)
X["test"], y["test"] = get_data(test_generator, do_repeat_relevant=False)
X["test_A"], y["test_A"] = get_data(test_generator_A, do_repeat_relevant=False)

In [0]:
key = "test"
print(len(X[key]))
print(len(y[key]))
for i in range(5):
  x = X[key][i]
  print(x[0].id + " " + x[0].body)
  print(x[1].metadata.id + " " + x[1].body)
#print(X[0][0])
print(type(y[key]))
print(y[key][:5])

8800
8800
Q388_R14 Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388_R14_C1 standard is 2 weeks maximum, but when I apply my schengen visa from Hungarian Embassy took 5 weeks.
Q388_R14 Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388_R14_C2 Thanks Novita for your quick reply.
Q388_R14 Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388_R14_C3 Hungary part of schengen ? ----------------- HE WHO DARES WINS
Q388_R14 Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388_R14_C4 Yes UKEng, only joined recently. I think that is why their visa application process took ages.
Q388_R14 Do you know how long it will take to get Schengen Visa from Embassy of Greece? 3 days? 1 week? any idea? Thanks.
Q388_R14_C5 I know that have recently 

In [0]:
questions = {"train": [], "dev":[], "test":[], "test_A":[]}
answers = {"train": [], "dev":[], "test":[], "test_A":[]}
for key in X.keys():
  for x in X[key]:
    questions[key].append(x[0].body)
    answers[key].append(x[1].body)
  
#print(questions[:5])
#print(answers[:5])

tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(questions["train"] + answers["train"])
vocab_size = len(tokenizer.word_index)
print('Found %s unique tokens.' % vocab_size)

quest = {"train": [], "dev":[], "test":[], "test_A":[]}
answ = {"train": [], "dev":[], "test":[], "test_A":[]}

# integer encode the sentences
for key in questions.keys():
  quest[key] = tokenizer.texts_to_sequences(questions[key])
  
for key in answers.keys():
  answ[key] = tokenizer.texts_to_sequences(answers[key])

# prepare data for Keras network
MAX_SEQUENCE_LENGTH = 100

train_X = [pad_sequences(quest["train"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["train"], maxlen=MAX_SEQUENCE_LENGTH)]
dev_X = [pad_sequences(quest["dev"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["dev"], maxlen=MAX_SEQUENCE_LENGTH)]
test_X = [pad_sequences(quest["test"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["test"], maxlen=MAX_SEQUENCE_LENGTH)]
test_A_X = [pad_sequences(quest["test_A"], maxlen=MAX_SEQUENCE_LENGTH), pad_sequences(answ["test_A"], maxlen=MAX_SEQUENCE_LENGTH)]

train_y = np.array(y["train"])
dev_y = np.array(y["dev"])
test_y = np.array(y["test"])
test_A_y = np.array(y["test_A"])


Found 30786 unique tokens.


In [0]:
print(test_X[0][:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   37    9   36   44  164   11   21
   115    2   38 6470   39   31  427   10 6598  103  137  120  271   30
   207   47]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   37    9   36   44  164   11   21
   115    2   38 6470   39   31  427   10 6598  103

In [0]:
maxlen = 100

train_lens = [np.count_nonzero(t) for t in answ["train"] + quest["train"]]
x = len(train_lens)
sx = len([l for l in train_lens if l > maxlen])
print("Train sentences longer than {} tokens: {:.2f} %".format(maxlen,sx/x*100))

Train sentences longer than 100 tokens: 3.69 %


In [0]:
print(train_X[0][:5])
print(train_X[1][:5])
print(train_y[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    7   28   30  123    3   15   93 7568  927 2755
     6   27]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    7   28   30  123    3   15

## GloVe

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
    
print('Embedding matrix shape=', embedding_matrix.shape)

Loaded 400000 word vectors.
Embedding matrix shape= (30787, 300)


In [0]:
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def myLSTM(*args, **kwargs):
  if device_name == '/device:GPU:0':
    return CuDNNLSTM(*args, **kwargs)
  return LSTM(*args, **kwargs)

def myGRU(*args, **kwargs):
  if device_name == '/device:GPU:0':
    return CuDNNGRU(*args, **kwargs)
  return GRU(*args, **kwargs)

Found GPU at: /device:GPU:0


## Final CNN model

In [0]:
#
# CNN classifer
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# convolution one
conv1 = Conv1D(filters, kernel_size, name='conv1', padding='valid', activation=activation, strides=1)
conv1_q = conv1(embed_q)
conv1_a = conv1(embed_a)

pool1 = GlobalMaxPooling1D(name='pool1')
pool1_q = pool1(conv1_q)
pool1_a = pool1(conv1_a)

# convolution two
conv2 = Conv1D(filters, kernel_size, name='conv2', padding='valid', activation=activation, strides=1)
conv2_q = conv2(conv1_q)
conv2_a = conv2(conv1_a)

pool2 = GlobalMaxPooling1D(name='pool2')
pool2_q = pool2(conv2_q)
pool2_a = pool2(conv2_a)

# convolution three
conv3 = Conv1D(filters, kernel_size, name='conv3', padding='valid', activation=activation, strides=1)
conv3_q = conv1(conv2_q)
conv3_a = conv1(conv2_a)

pool3 = GlobalMaxPooling1D(name='pool3')
pool3_q = pool1(conv3_q)
pool3_a = pool1(conv3_a)

# convolution four
conv4 = Conv1D(filters, kernel_size, name='conv4', padding='valid', activation=activation, strides=1)
conv4_q = conv1(conv3_q)
conv4_a = conv1(conv3_a)

pool4 = GlobalMaxPooling1D(name='pool4')
pool4_q = pool1(conv4_q)
pool4_a = pool1(conv4_a)

# concatenations
q_ = concatenate([pool1_q, pool2_q, pool3_q, pool4_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_a, pool3_a, pool4_a], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

CNN = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
CNN.compile(loss='binary_crossentropy',
              optimizer='adadelta',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(CNN.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     9236100     masking[0][0]                    
                                                                 masking[1][0]                    
______________________________________________________________________________________________

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*3#10*2

print('Train...')
CNN.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = CNN.evaluate(test_A_X, test_A_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
CNN.save('msa_CNN_new.h5')

Train...
Train on 26690 samples, validate on 5000 samples
Epoch 1/30
 - 47s - loss: 0.6405 - acc: 0.6477 - val_loss: 0.6103 - val_acc: 0.6626
Epoch 2/30
 - 44s - loss: 0.5744 - acc: 0.7130 - val_loss: 0.5523 - val_acc: 0.7198
Epoch 3/30
 - 44s - loss: 0.5292 - acc: 0.7384 - val_loss: 0.5792 - val_acc: 0.7100
Epoch 4/30
 - 44s - loss: 0.5039 - acc: 0.7558 - val_loss: 0.5878 - val_acc: 0.7086
Epoch 5/30
 - 44s - loss: 0.4682 - acc: 0.7758 - val_loss: 0.5668 - val_acc: 0.7084
Epoch 6/30
 - 44s - loss: 0.4486 - acc: 0.7860 - val_loss: 0.5190 - val_acc: 0.7506
Epoch 7/30
 - 44s - loss: 0.4270 - acc: 0.7987 - val_loss: 0.6295 - val_acc: 0.7086
Epoch 8/30
 - 44s - loss: 0.4090 - acc: 0.8102 - val_loss: 0.5210 - val_acc: 0.7478
Epoch 9/30
 - 44s - loss: 0.3934 - acc: 0.8199 - val_loss: 0.5246 - val_acc: 0.7498
Epoch 10/30
 - 44s - loss: 0.3724 - acc: 0.8299 - val_loss: 0.6195 - val_acc: 0.7362
Epoch 11/30
 - 44s - loss: 0.3548 - acc: 0.8408 - val_loss: 0.5351 - val_acc: 0.7428
Epoch 12/30
 - 4

2930/2930 [==============================] - 1s 498us/step
Test score: 1.0896345565343473
Test accuracy: 0.6556313992360346


In [0]:
predict_and_save(CNN, test_A_X, X["test_A"], "preds/cnn_msa_new.pred")

In [0]:
files.download('msa_CNN_new.h5')

In [0]:
#
# CNN classifer average pooling
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# convolution one
conv1 = Conv1D(filters, kernel_size, name='conv1', padding='valid', activation=activation, strides=1)
conv1_q = conv1(embed_q)
conv1_a = conv1(embed_a)

pool1 = GlobalAveragePooling1D(name='pool1')
pool1_q = pool1(conv1_q)
pool1_a = pool1(conv1_a)

# convolution two
conv2 = Conv1D(filters, kernel_size, name='conv2', padding='valid', activation=activation, strides=1)
conv2_q = conv2(conv1_q)
conv2_a = conv2(conv1_a)

pool2 = GlobalAveragePooling1D(name='pool2')
pool2_q = pool2(conv2_q)
pool2_a = pool2(conv2_a)

# convolution three
conv3 = Conv1D(filters, kernel_size, name='conv3', padding='valid', activation=activation, strides=1)
conv3_q = conv1(conv2_q)
conv3_a = conv1(conv2_a)

pool3 = GlobalAveragePooling1D(name='pool3')
pool3_q = pool1(conv3_q)
pool3_a = pool1(conv3_a)

# convolution four
conv4 = Conv1D(filters, kernel_size, name='conv4', padding='valid', activation=activation, strides=1)
conv4_q = conv1(conv3_q)
conv4_a = conv1(conv3_a)

pool4 = GlobalAveragePooling1D(name='pool4')
pool4_q = pool1(conv4_q)
pool4_a = pool1(conv4_a)

# concatenations
q_ = concatenate([pool1_q, pool2_q, pool3_q, pool4_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_a, pool3_a, pool4_a], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

CNN1 = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
CNN1.compile(loss='binary_crossentropy',
              optimizer='adadelta',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(CNN1.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     9236100     masking[0][0]                    
                                                                 masking[1][0]                    
______________________________________________________________________________________________

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*3

print('Train...')
CNN1.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = CNN.evaluate(test_A_X, test_A_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
CNN1.save('msa_CNN1.h5')

Train...
Train on 26690 samples, validate on 5000 samples
Epoch 1/30
 - 46s - loss: 0.6603 - acc: 0.6365 - val_loss: 0.6627 - val_acc: 0.6266
Epoch 2/30
 - 43s - loss: 0.6149 - acc: 0.6803 - val_loss: 0.6250 - val_acc: 0.6634
Epoch 3/30
 - 43s - loss: 0.5909 - acc: 0.7015 - val_loss: 0.5848 - val_acc: 0.6948
Epoch 4/30
 - 43s - loss: 0.5748 - acc: 0.7114 - val_loss: 0.6550 - val_acc: 0.6546
Epoch 5/30
 - 43s - loss: 0.5546 - acc: 0.7266 - val_loss: 0.5646 - val_acc: 0.7030
Epoch 6/30
 - 43s - loss: 0.5381 - acc: 0.7402 - val_loss: 0.5858 - val_acc: 0.7002
Epoch 7/30
 - 43s - loss: 0.5209 - acc: 0.7492 - val_loss: 0.5477 - val_acc: 0.7132
Epoch 8/30
 - 43s - loss: 0.5038 - acc: 0.7592 - val_loss: 0.6124 - val_acc: 0.6768
Epoch 9/30
 - 43s - loss: 0.4903 - acc: 0.7704 - val_loss: 0.5225 - val_acc: 0.7398
Epoch 10/30
 - 43s - loss: 0.4964 - acc: 0.7657 - val_loss: 0.5674 - val_acc: 0.7074
Epoch 11/30
 - 43s - loss: 0.4674 - acc: 0.7819 - val_loss: 0.5367 - val_acc: 0.7448
Epoch 12/30
 - 4

2930/2930 [==============================] - 1s 493us/step
Test score: 0.6978245946531003
Test accuracy: 0.7051194538028573


In [0]:
predict_and_save(CNN1, test_A_X, X["test_A"], "preds/cnn_msa1.pred")

In [0]:
files.download('msa_CNN1.h5')

## Final BiGRU model

In [0]:
#
# CNN classifer
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# biGRU one
gru_q1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_q)
gru_a1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_a)

pool1 = GlobalMaxPooling1D(name='pool1')
pool1_q = pool1(gru_q1)
pool1_a = pool1(gru_a1)

# biGRU two
gru_q2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_q1)
gru_a2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_a1)

pool2 = GlobalMaxPooling1D(name='pool2')
pool2_q = pool2(gru_q1)
pool2_a = pool2(gru_a1)

# concatenations
q_ = concatenate([pool1_q, pool2_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_q], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

BiGRU = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
BiGRU.compile(loss='binary_crossentropy',
              optimizer='adadelta',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(BiGRU.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
answer (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     9236100     masking[0][0]                

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*2

print('Train...')
BiGRU.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = BiGRU.evaluate(test_A_X, test_A_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
BiGRU.save('msa_BiGRU.h5')

Train...
Train on 26690 samples, validate on 5000 samples
Epoch 1/20
 - 113s - loss: 0.6533 - acc: 0.6320 - val_loss: 0.6241 - val_acc: 0.6472
Epoch 2/20
 - 110s - loss: 0.5759 - acc: 0.7047 - val_loss: 0.6916 - val_acc: 0.6480
Epoch 3/20
 - 110s - loss: 0.5105 - acc: 0.7535 - val_loss: 0.6006 - val_acc: 0.6760
Epoch 4/20
 - 110s - loss: 0.4608 - acc: 0.7846 - val_loss: 0.5046 - val_acc: 0.7592
Epoch 5/20
 - 110s - loss: 0.4216 - acc: 0.8105 - val_loss: 0.4906 - val_acc: 0.7624
Epoch 6/20
 - 110s - loss: 0.3837 - acc: 0.8289 - val_loss: 0.4827 - val_acc: 0.7802
Epoch 7/20
 - 110s - loss: 0.3556 - acc: 0.8468 - val_loss: 0.4628 - val_acc: 0.7820
Epoch 8/20
 - 110s - loss: 0.3280 - acc: 0.8576 - val_loss: 0.4648 - val_acc: 0.7868
Epoch 9/20
 - 110s - loss: 0.3070 - acc: 0.8715 - val_loss: 0.6517 - val_acc: 0.7120
Epoch 10/20
 - 110s - loss: 0.2848 - acc: 0.8794 - val_loss: 0.5322 - val_acc: 0.7818
Epoch 11/20
 - 110s - loss: 0.2639 - acc: 0.8912 - val_loss: 0.6684 - val_acc: 0.7172
Epoch

2930/2930 [==============================] - 4s 1ms/step
Test score: 0.9554646682820629
Test accuracy: 0.654266211522724


In [0]:
predict_and_save(BiGRU, test_A_X, X["test_A"], "preds/bigru_msa.pred")

In [0]:
files.download('msa_BiGRU.h5')

In [0]:
#
# BiGRU average pooling
#
from keras.optimizers import *
sgd = SGD(lr=0.1, momentum=0.1, decay=0.99, nesterov=True)
#adagrad = Adagrad(lr=0.5, epsilon=None, decay=0.09)
activation = 'elu'
padding = 'valid'
filters = 300
kernel_size = 3

HIDDEN = 300

print('Build model...')
inputs_q = Input(shape=(MAX_SEQUENCE_LENGTH,), name='question')
inputs_a = Input(shape=(MAX_SEQUENCE_LENGTH,), name='answer')

# add masking
mask = Masking(mask_value=0., name='masking')
inputs_q_ = mask(inputs_q)
inputs_a_ = mask(inputs_a)

embed = Embedding(vocab_size+1, HIDDEN, weights=[embedding_matrix], trainable=False, mask_zero=False, name='embedding')#mask_zero=True,trainable=False
embed_q = embed(inputs_q_)
embed_a = embed(inputs_a_)

# biGRU one
gru_q1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_q)
gru_a1 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(embed_a)

pool1 = GlobalAveragePooling1D(name='pool1')
pool1_q = pool1(gru_q1)
pool1_a = pool1(gru_a1)

# biGRU two
gru_q2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_q1)
gru_a2 = Bidirectional(myGRU(HIDDEN, return_sequences=True), merge_mode="concat")(gru_a1)

pool2 = GlobalAveragePooling1D(name='pool2')
pool2_q = pool2(gru_q1)
pool2_a = pool2(gru_a1)

# concatenations
q_ = concatenate([pool1_q, pool2_q], name='question_concat')
a_ = concatenate([pool1_a, pool2_q], name='answer_concat')
re_q = Reshape((-1, HIDDEN))(q_)
re_a = Reshape((-1, HIDDEN))(a_)
q = Dropout(rate=0.25, name='question_dropout')(re_q)#(q_)
a = Dropout(rate=0.25, name='answer_dropout')(re_a)#(a_)

qa_mul = multiply([q, a], name='multiply') 
qa_sub = subtract([q, a], name='subtract')
qa_max = maximum([q, a], name='maximum')
qa_avg = average([q, a], name='average')
concat = concatenate([q, a, qa_max, qa_mul, qa_sub, qa_avg], name='final_concat')

# attention block - multi-dimensional self-attention
reshaped = Reshape((-1, HIDDEN))(concat)
compatibilities = Dense(HIDDEN, activation='elu', use_bias=False, name='compatibilites')(reshaped)
attention_weights = Dense(HIDDEN, activation='softmax', use_bias=False, name='self_att_weights')(compatibilities)
attention_ = multiply([reshaped, attention_weights], name='attention_mul')
attention = Lambda(lambda x: K.sum(x, axis=1), name="attention")(attention_)
#attention = dot([reshaped, attention_weights], axes=-1, name='attention')

hidden_ = Dense(HIDDEN, activation='tanh', name='final_dense_1')(attention)
#hidden_2 = Dense(HIDDEN, activation='elu', name='final_dense_2')(hidden_1)
#hidden_ = Dropout(rate=0.25, name="hidden_dropout")(hidden_2)
#hidden = BatchNormalization()(hidden_)

preds = Dense(1, activation='sigmoid', name='predictions')(attention)#(hidden_)

BiGRU1 = Model(inputs=[inputs_q, inputs_a], outputs=[preds])
# try using different optimizers and different optimizer configs
BiGRU1.compile(loss='binary_crossentropy',
              optimizer='adadelta',#'adadelta',
              metrics=['accuracy']) # add mean_average_precision ?
print(BiGRU1.summary())

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
answer (InputLayer)             (None, 100)          0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, 100)          0           question[0][0]                   
                                                                 answer[0][0]                     
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     9236100     masking[0][0]                

In [0]:
BATCH_SIZE = 32 # set to max allowable value
EPOCHS = 10*2

print('Train...')
BiGRU1.fit(train_X, train_y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(dev_X, dev_y),
          verbose=2,
          callbacks=None)

score, acc = BiGRU1.evaluate(test_A_X, test_A_y, batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)
# save model
BiGRU1.save('msa_BiGRU1.h5')

Train...
Train on 26690 samples, validate on 5000 samples
Epoch 1/20
 - 113s - loss: 0.6638 - acc: 0.6251 - val_loss: 0.6555 - val_acc: 0.6298
Epoch 2/20
 - 110s - loss: 0.6360 - acc: 0.6478 - val_loss: 0.6495 - val_acc: 0.6420
Epoch 3/20
 - 110s - loss: 0.6033 - acc: 0.6819 - val_loss: 0.6308 - val_acc: 0.6534
Epoch 4/20
 - 110s - loss: 0.5663 - acc: 0.7156 - val_loss: 0.5932 - val_acc: 0.6840
Epoch 5/20
 - 110s - loss: 0.5513 - acc: 0.7268 - val_loss: 0.5457 - val_acc: 0.7320
Epoch 6/20
 - 110s - loss: 0.5272 - acc: 0.7448 - val_loss: 0.5859 - val_acc: 0.7022
Epoch 7/20
 - 110s - loss: 0.5034 - acc: 0.7588 - val_loss: 0.5358 - val_acc: 0.7356
Epoch 8/20
 - 110s - loss: 0.4842 - acc: 0.7725 - val_loss: 0.6159 - val_acc: 0.7134
Epoch 9/20
 - 110s - loss: 0.4674 - acc: 0.7854 - val_loss: 0.5422 - val_acc: 0.7472
Epoch 10/20
 - 110s - loss: 0.4470 - acc: 0.7977 - val_loss: 0.5338 - val_acc: 0.7534
Epoch 11/20
 - 110s - loss: 0.4320 - acc: 0.8074 - val_loss: 0.5467 - val_acc: 0.7522
Epoch

2930/2930 [==============================] - 4s 1ms/step
Test score: 0.7577424345773235
Test accuracy: 0.6757679181701087


In [0]:
predict_and_save(BiGRU1, test_A_X, X["test_A"], "preds/bigru_msa1.pred")

In [0]:
files.download('msa_BiGRU1.h5')

## Evaluation

In [0]:
X_test, y_test = X["test"], y["test"]

In [0]:
def predict(model, inputs, batch_size):
    probs = model.predict(inputs, batch_size=batch_size)
    return np.squeeze(probs), np.array(["false" if x < 0.5 else "true" for x in probs])

In [0]:
!mkdir preds
!ls

checkpoints  datalab  glove.zip  task3-cqa.zip	test.zip
data	     glove    preds	 test_data


In [0]:
def create_pred_file(probs, preds, X, pred_filepath):
    with open(pred_filepath, 'w') as pred_file:
        for i in range(len(X)):
            x = X[i]
            print("{}\t{}\t{}\t{}\t{}".format(x[0].id, x[1].metadata.id, 0, probs[i], preds[i]), 
                  file=pred_file)
    files.download(pred_filepath)
    

def predict_and_save(model, processed_input, unprocessed_input, pred_filepath):
    probs, preds = predict(model, processed_input, 32)
    create_pred_file(probs, preds, unprocessed_input, pred_filepath)

In [0]:
predict_and_save(abl_model, test_A_X, X["test_A"], "preds/abl_model_nomax.pred")

In [0]:
!ls preds

bilstm_max.pred


In [0]:
files.download("preds/bilstm_max.pred")

In [0]:
!ls preds
!head -n 5 preds/bilstm_max.pred

bilstm_max.pred
Q388_R14	Q388_R14_C1	0	0.944097638130188	true
Q388_R14	Q388_R14_C2	0	0.01067881565541029	false
Q388_R14	Q388_R14_C3	0	0.1160203218460083	false
Q388_R14	Q388_R14_C4	0	0.25709769129753113	false
Q388_R14	Q388_R14_C5	0	0.1618800163269043	false


## Demo

In [0]:
import textwrap

def rank_anwsers_for_question(model, data, proc_data, q_index):
    ind = q_index * 10
    example = data[ind : ind+10]
    print("QUESTION")
    print("\n".join(textwrap.wrap(example[0][0].body, 120, break_long_words=False)))
    print("\nANSWERS")
    for i in range(10):
        line = "{}) {}".format(i+1, example[i][1].body)
        print("\n".join(textwrap.wrap(line, 120, break_long_words=False)))
        print()
      
    proc_example_q = proc_data[0][ind:ind+10, :]
    proc_example_a = proc_data[1][ind:ind+10, :]
    proc_example = [proc_example_q, proc_example_a]
    
    probs, preds = predict(model, proc_example, 10)
    probs_preds = list(zip(probs, preds))
    ranked_probs_preds = sorted(enumerate(probs_preds), key=lambda x: -x[1][0])
    
    ranked_answers = []
    for rpp in ranked_probs_preds:
        idx = rpp[0]
        ranked_answers.append((example[idx][1].body, rpp[1][1]))
      
    print("\nRANKED ANSWERS")
    for i in range(len(ranked_answers)):
        line = "{}) {}".format(i+1, ranked_answers[i][0])
        print("\n".join(textwrap.wrap(line, 120, break_long_words=False)))
        print("IS RELEVANT? {}".format(ranked_answers[i][1]))
        print()

In [0]:
rank_anwsers_for_question(bilstm_max_demo, X["test_A"], test_A_X, 10)

QUESTION
hi all i arrive here with the sponsor of my husband with visit visa.i get entry visa for 2months.and stay of 1 month. i
can extend stay till 6 months. i am hunting for a job here in qatar. the companyn that i am applying for a job said to
apply for those in family visa. so what's the difference between my visa and family visa? of is it the same? pls answer
me bk ASAP. thanks...

ANSWERS
1) hi there! just some clarifications...i know those who are in family visit visa have the privilege for a 1 month stay
and can get it extended for 5 months. Im quite confused with the 2 months entry and 1 month stay...? extension
maybe?..that might be a business visa i guess. Husband sponsored visa is entirely different because your validity of
stay in qatar depends totally on your husband visa status...? Maybe what the company meant was they accept applicants or
workers only for those with husband visa sponsorship...Anyway look for company which can provide a working visa, if you
do have more